It is an Implementation of Shor's Algorithm

In [1]:
import numpy as np
import random
from math import gcd as mathgcd, sqrt, log2, ceil
from fractions import Fraction
from functools import lru_cache

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import QuantumCircuit
from quantumrings.toolkit.qiskit import QrBackendV2, QrJobV1
from QuantumRingsLib import QuantumRingsProvider, job_monitor

provider = QuantumRingsProvider(
    token='rings-128.YXZRFUMMfK0iaD7TlGMbb9Z2qUh3XrBC',
    name='aryan.malhotra@rutgers.edu'
)
backend = provider.get_backend("scarlet_quantum_rings")


In [13]:
########################################################
# Helper Functions
########################################################
def generate_guess_pow2(N):
    """Guess based on half the binary length of N."""
    hl = (len(bin(N)) - 2) // 2
    return int("1" + "0" * hl, 2)

def generate_guess_random(N):
    """Random guess up to sqrt(N)."""
    while True:
        a = random.randint(2, round(sqrt(N)))
        g = mathgcd(a, N)
        if g == 1:
            return a, True
        else:
            return g, False

def generate_guess(N, itr=0):
    """
    1) Try generate_guess_pow2
    2) If that guess yields gcd != 1, return that factor.
    3) Otherwise, switch to generate_guess_random
    """

    if itr == -1:
        return generate_guess_pow2(N), True
    else:
        return generate_guess_random(N)


########################################################
# Simplified Controlled Modular Operations
########################################################
def controlled_mod_mult(qc, a, N, control_qubit, target_qubits, ancilla_qubits):
    """Small-N demonstration for (a*x) % N with ancillas."""
    num_target_qubits = len(target_qubits)
    for anc in ancilla_qubits:
        qc.reset(anc)
    for i in range(num_target_qubits):
        for j in range(num_target_qubits):
            if (a * (2**i) * (2**j)) % N != 0:
                qc.ccx(control_qubit, target_qubits[j], ancilla_qubits[i])
    for i in range(num_target_qubits):
        qc.cx(ancilla_qubits[i], target_qubits[i])
    for anc in ancilla_qubits:
        qc.reset(anc)

def controlled_mod_exp(qc, a, N, control_qubit, target_qubits, ancilla_qubits):
    """Build a^(2^i) mod N in the target."""
    num_target_qubits = len(target_qubits)
    for i in range(num_target_qubits):
        base_exp = pow(a, 2**i, N)
        controlled_mod_mult(qc, base_exp, N, control_qubit, target_qubits, ancilla_qubits)
        
########################################################
# Inverse QFT (for small N)
########################################################
def create_inverse_qft_circuit(n):
    from QuantumRingsLib import QuantumCircuit
    qc = QuantumCircuit(n)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            angle = -np.pi / (2 ** (i - j))
            qc.cu1(angle, j, i)
        qc.h(i)
    return qc

########################################################
# Phase Estimation (QPEN) & Order Extraction
########################################################
@lru_cache(maxsize=None)
def QPEN(a, N):
    num_qubits = int(ceil(log2(N))) + 2
    total_qubits = num_qubits*2 + 4  # Double the num_qubits for phase and target registers
    # print(total_qubits)
    qc = QuantumCircuit(total_qubits, num_qubits) # Initialize with total_qubits and num_qubits classical bits

    # Hadamards on phase qubits
    for bit in range(num_qubits):
        qc.h(bit)

    # Prepare target: set one qubit to |1>
    qc.x(num_qubits + 1)

    # Controlled exponentiation
    for phase_bit in range(num_qubits):
        controlled_mod_exp(qc, a, N, phase_bit,
                           list(range(num_qubits, num_qubits*2)),
                           list(range(num_qubits*2, num_qubits*2 + 4)))

    # Inverse QFT
    iqft_circ = create_inverse_qft_circuit(num_qubits)
    qc.append(iqft_circ, range(num_qubits))

    # Measure
    qc.measure(range(num_qubits), range(num_qubits))

    # Execute on QuantumRings
    from qiskit_ibm_runtime import QiskitRuntimeService


    job = backend.run(qc, shots=1024)
    job_monitor(job)  # Optionally track progress

    result = job.result()
    counts = result.get_counts()

    # Convert bitstrings to fractional phases
    phases = []
    for out_str, count_val in counts.items():
        decimal_val = int(out_str, 2)
        phase_val = decimal_val / (2**num_qubits)
        phases.append(phase_val)

    #clean up
    del qc
    del result
    del job
    
    return phases

def r_of_a_mod_N(a, N):
    phases = QPEN(a, N)
    frac_list = [Fraction(ph).limit_denominator(N) for ph in phases]
    orders = []
    for f_num in frac_list:
        if f_num.denominator not in (1, *orders):
            orders.append(f_num.denominator)
    return orders

########################################################
# Final Shor Algorithm with fallback logic
########################################################
def shors_algorithm(N):

    # Attempt up to 10000 guesses
    for _ in range(N*int(log2(N))):
        # print("test")
        a, relatively_prime = generate_guess(N, _)
        if not relatively_prime:
            print(f"Found factors: {a}, {N // a}")
            return a, N // a, True
        else:
            # print(f"Candidate: a={a}")
            pass
         
        
        try:
            # Do quantum phase estimation and interpret order
            orders = r_of_a_mod_N(a, N)
            if orders:
                r = orders[0]
                if r % 2 == 0:
                    print(f"Candidate: a={a}, r={r}")
                    factor1 = np.gcd(pow(a, r // 2, N) - 1, N)
                    factor2 = np.gcd(pow(a, r // 2, N) + 1, N)
                    if factor1 > 1 and factor2 > 1:
                        print(f"Found factors: {factor1}, {factor2}")
                        return factor1, factor2, True
        except Exception:
            pass
                
    return None

In [14]:
def run_shors_algorithm_until_success(N):
    """
    Run Shor's algorithm until it completes successfully without throwing an exception.
    
    Parameters:
    - N: The number to factorize
    - max_retries: Maximum number of retries
    
    Returns:
    - factors: The factors found by Shor's algorithm
    """
    print(f"Attempting to factor {N}...")
    maxa= N*int(log2(N))
    attempt = 0
    while attempt < maxa:
        try:
            factors = shors_algorithm(N)
            return factors
        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed: {e}")
            if attempt >= maxa:
                raise RuntimeError("Max retries exceeded")
            print("Retrying...")


In [16]:
# Example usage
N = 857830637
factors = run_shors_algorithm_until_success(N)
print("###################################################")
print(f"Factors of {N} are: {factors}")

Attempting to factor 857830637...
Found factors: 29167, 29411
###################################################
Factors of 857830637 are: (29167, 29411, True)
